In [3]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import sqlite3
import csv
import os
import sys
import json

%matplotlib inline
N = 10
to_id = lambda x: "0"+str(x) if len(str(x)) == 1 else str(x);
IDS = map(to_id, xrange(N))

#debug
# N = 1
# IDS = ['00']
# DEBUG_ID = '00'

print IDS

['00', '01', '02', '03', '04', '05', '06', '07', '08', '09']


### Sensor Features

In [4]:
__dir = "./dataset"
sensing_dir = __dir + "/sensing"
sensing_types = ['activity', 'audio', 'bluetooth', 'conversation',
                 'dark', 'gps', 'phonecharge', 'phonelock', 'wifi', 'wifi_location']

data = {}
for _id in IDS:
    data[_id] = {}
    for _type in sensing_types:
        print _id, _type
        data[_id][_type] = [];
        fname = "%s/%s/%s_u%s.csv"%(sensing_dir, _type, _type,_id)
#         print fname, os.path.exists(fname)
        if os.path.exists(fname):
            with open(fname) as csvf:
                csv_reader = csv.reader(csvf, delimiter=',')
                is_header = True
                for row in csv_reader:                    
                    if is_header or len(row)<2: 
                        is_header = False
                        continue
                    data[_id][_type].append(row)
    
print len(data)


00 activity
00 audio
00 bluetooth
00 conversation
00 dark
00 gps
00 phonecharge
00 phonelock
00 wifi
00 wifi_location
01 activity
01 audio
01 bluetooth
01 conversation
01 dark
01 gps
01 phonecharge
01 phonelock
01 wifi
01 wifi_location
02 activity
02 audio
02 bluetooth
02 conversation
02 dark
02 gps
02 phonecharge
02 phonelock
02 wifi
02 wifi_location
03 activity
03 audio
03 bluetooth
03 conversation
03 dark
03 gps
03 phonecharge
03 phonelock
03 wifi
03 wifi_location
04 activity
04 audio
04 bluetooth
04 conversation
04 dark
04 gps
04 phonecharge
04 phonelock
04 wifi
04 wifi_location
05 activity
05 audio
05 bluetooth
05 conversation
05 dark
05 gps
05 phonecharge
05 phonelock
05 wifi
05 wifi_location
06 activity
06 audio
06 bluetooth
06 conversation
06 dark
06 gps
06 phonecharge
06 phonelock
06 wifi
06 wifi_location
07 activity
07 audio
07 bluetooth
07 conversation
07 dark
07 gps
07 phonecharge
07 phonelock
07 wifi
07 wifi_location
08 activity
08 audio
08 bluetooth
08 conversation
08 dar

In [ ]:
# for _id in IDS:
#     for _type in sensing_types:
#         if len(data[_id][_type]) == 0: 
#             print _type, 'null'
#             continue
#         print _type, data[_id][_type][0]
#     break;
print data['00'].keys()


### EMA features
stress, sleep, mood

In [ ]:
ema_dir = __dir + '/EMA/response'
ema_types = ['Stress', 'Activity', 'Mood', 'Mood 1', 'Mood 2', 'Exercise']
ema_data = {}
TIME_KEY = "resp_time"

for _id in IDS:
    ema_data[_id] = {}
    for _type in ema_types:
        fname= '%s/%s/%s_u%s.json'%(ema_dir, _type, _type, _id)
        ema_data[_id][_type] = []
        if (os.path.exists(fname)):
            with open(fname) as json_file:
                json_array = json.load(json_file)
#                 print type, json_obj[0]
                for json_obj in json_array:
                    ema_data[_id][_type].append(json_obj)
                    
                

### Missing Data

In [10]:
# Sensor data

missing_data = {}
interval = 5*60. #in seconds
for _id in IDS:
    missing_data[_id] = {}
    for _type in data[_id].keys():
        min_time = sys.maxint
        max_time = -sys.maxint
        features_ts = data[_id][_type]
        missing_cnt = 0
        print _type, len(features_ts)
        for i in range(1, len(features_ts)):
            datum = features_ts[i]
            _prev_datum = features_ts[i-1]
            
            _time = int(datum[0])
            _prev_time = int(_prev_datum[0])
            
            min_time = min(min_time, _time)
            max_time = max(max_time, _time)
            min_time = min(min_time, _prev_time)
            max_time = max(max_time, _prev_time)
            
            missing = abs(_time - _prev_time) / interval
            if (missing < 1): missing = 0
            else: missing -= 1
#             print _time, _prev_time, _time > _prev_time, missing
            missing_cnt += (missing)
        total_cnt = (max_time - min_time) / interval
        missing_p = missing_cnt/total_cnt
        missing_data[_id][_type] = {'hi': max_time, 'lo': min_time, 
                                    'missing': missing_cnt, 'total': total_cnt,
                                    'interval': interval, 'missing_p': missing_p,
                                    'length': len(features_ts)}
#         print '%s / %s || %.3f' %(int(missing_cnt), int(total_cnt), float(missing_p))
print len(missing_data)


wifi_location 11221
wifi 446110
bluetooth 0
dark 184
conversation 2111
phonecharge 75
activity 461991
phonelock 226
audio 2549558
gps 4316
wifi_location 20149
wifi 164662
bluetooth 0
dark 205
conversation 2179
phonecharge 44
activity 474862
phonelock 216
audio 2608482
gps 5541
wifi_location 27947
wifi 331058
bluetooth 0
dark 187
conversation 1483
phonecharge 105
activity 527518
phonelock 156
audio 2444394
gps 6607
wifi_location 15251
wifi 136429
bluetooth 0
dark 63
conversation 478
phonecharge 47
activity 411402
phonelock 102
audio 1311046
gps 3502
wifi_location 30811
wifi 407699
bluetooth 0
dark 268
conversation 2737
phonecharge 60
activity 462516
phonelock 241
audio 2456731
gps 3620
wifi_location 23058
wifi 214136
bluetooth 0
dark 170
conversation 1640
phonecharge 72
activity 471647
phonelock 180
audio 2133762
gps 5388
wifi_location 0
wifi 0
bluetooth 0
dark 0
conversation 0
phonecharge 0
activity 0
phonelock 0
audio 0
gps 0
wifi_location 57202
wifi 603870
bluetooth 0
dark 152
conver

In [12]:
sensor_avg_missing = {}
for _type in sensing_types:
    sensor_avg_missing[_type] = 0.0
    for _id in data.keys():
        sensor_avg_missing[_type] += missing_data[_id][_type]['missing_p']
    sensor_avg_missing[_type] /= N
    
print sensor_avg_missing

{'wifi_location': 0.4221004480870308, 'wifi': 0.34273186711497383, 'bluetooth': 0.0, 'dark': 0.8906946486776237, 'conversation': 0.8061316614346428, 'phonecharge': 0.8964251592955483, 'activity': 0.057478044091534966, 'phonelock': 0.8899035139754161, 'audio': 0.0574338618490798, 'gps': 0.6577509046934501}


In [ ]:
missing_data = {}
interval = 3600. #in seconds
for _id in IDS:
    missing_data[_id] = {}
    for _type in data[_id].keys():
        min_time = sys.maxint
        max_time = -sys.maxint
        features_ts = data[_id][_type]
        missing_cnt = 0
        print _type, len(features_ts)
        for i in range(1, len(features_ts)):
            datum = features_ts[i]
            _prev_datum = features_ts[i-1]
            
            _time = int(datum[0])
            _prev_time = int(_prev_datum[0])
            
            min_time = min(min_time, _time)
            max_time = max(max_time, _time)
            min_time = min(min_time, _prev_time)
            max_time = max(max_time, _prev_time)
            
            missing = abs(_time - _prev_time) / interval
            if (missing < 1): missing = 0
#             print _time, _prev_time, _time > _prev_time, missing
            missing_cnt += missing
        total_cnt = (max_time - min_time) / interval
        missing_p = missing_cnt/total_cnt
        missing_data[_id][_type] = {'hi': max_time, 'lo': min_time, 
                                    'missing': missing_cnt, 'total': total_cnt,
                                    'interval': interval, 'missing_p': missing_p}
        print '%s / %s || %.3f' %(int(missing_cnt), int(total_cnt), float(missing_p))